# Notebook to create `indicators` module

In [1]:
from pathlib import Path

PACKAGE = "mintalib"
ROOTDIR = Path.cwd().parent
PKGDIR = ROOTDIR.joinpath(f"src/{PACKAGE}").resolve(strict=True)

if '__file__' in globals():
    print(f"Running {__file__} ...")


In [2]:
from mintalib.samples import sample_prices

prices = sample_prices()
prices


,open,high,low,close,volume
date,,,,,
1980-12-12,0.098943,0.099373,0.098943,0.098943,469033600
1980-12-15,0.094211,0.094211,0.093781,0.093781,175884800
1980-12-16,0.087328,0.087328,0.086898,0.086898,105728000
1980-12-17,0.089049,0.089479,0.089049,0.089049,86441600
1980-12-18,0.091630,0.092061,0.091630,0.091630,73449600
...,...,...,...,...,...
2024-10-15,233.610001,237.490005,232.369995,233.850006,64751400
2024-10-16,231.600006,232.119995,229.839996,231.779999,34082200
2024-10-17,233.429993,233.850006,230.520004,232.149994,32993800


In [ ]:
PRELUDE='''"""
Indicators offer a composable interface where a calculation routine is bound with its parameters.

An indicator instance is a callable and can be applied to prices or series data as if it were a function e.g. `SMA(50)(prices)`.

Indicators also support the `@` operator to apply them to their input data e.g. `SMA(50) @ prices` or to chain them together e.g. `ROC(1) @ EMA(20)`.
"""

# Do not edit! This file was generated.

from mintalib import core
from mintalib.model.indicator import wrap_indicator

'''

exec(PRELUDE)

In [4]:
@wrap_indicator(core.calc_sma)
def SMA(period: int, *, item: str = None): ...

SMA(5) @ prices

date
1980-12-12           NaN
1980-12-15           NaN
1980-12-16           NaN
1980-12-17           NaN
1980-12-18      0.092060
                 ...    
2024-10-15    230.256000
2024-10-16    230.704001
2024-10-17    231.326001
2024-10-18    232.816000
2024-10-21    233.851999
Length: 11056, dtype: float64

In [5]:
import inspect

from mintalib.builder import annotate_parameter

def make_signature(calc_func):
    """creates function signature from core function"""
    
    sig = inspect.signature(calc_func)
    first_param = next(iter(sig.parameters.values()))
    
    new_params = []
    for param in sig.parameters.values():
        if param.name in ("series", "prices", "wrap"):
            continue
        param = annotate_parameter(param)
        new_params.append(param)

    if first_param.name == "series":
        item_param = inspect.Parameter(
            name="item",
            kind=inspect.Parameter.KEYWORD_ONLY,
            default=None,
            annotation=str
        )
        new_params.append(item_param)

    return sig.replace(parameters=new_params)

sig = make_signature(core.calc_sma)
print(sig)


(period: int, *, item: str = None)


In [6]:
def make_indicator(calc_func, name=None):
    if name is None:
        name = calc_func.__name__.removeprefix("calc_").upper()
    cname = f"core.{calc_func.__name__}"
    newsig = make_signature(calc_func)
    buffer = f"@wrap_indicator({cname})\n"
    buffer += f"def {name}{newsig}: ...\n"
    return buffer

output = make_indicator(core.calc_sma)
print(output)

@wrap_indicator(core.calc_sma)
def SMA(period: int, *, item: str = None): ...



In [7]:
def core_functions():
    return sorted(k for k, v in vars(core).items() if k.startswith("calc_") and callable(v))

core_functions()

['calc_abs',
 'calc_adx',
 'calc_alma',
 'calc_atr',
 'calc_avgprice',
 'calc_bbands',
 'calc_bbp',
 'calc_bbw',
 'calc_bop',
 'calc_cci',
 'calc_clag',
 'calc_cmf',
 'calc_crossover',
 'calc_crossunder',
 'calc_curve',
 'calc_dema',
 'calc_diff',
 'calc_dmi',
 'calc_ema',
 'calc_eval',
 'calc_exp',
 'calc_flag',
 'calc_hma',
 'calc_kama',
 'calc_keltner',
 'calc_ker',
 'calc_lag',
 'calc_log',
 'calc_lroc',
 'calc_macd',
 'calc_macdv',
 'calc_mad',
 'calc_mav',
 'calc_max',
 'calc_mdi',
 'calc_mfi',
 'calc_midprice',
 'calc_min',
 'calc_natr',
 'calc_pdi',
 'calc_ppo',
 'calc_price',
 'calc_qsf',
 'calc_rma',
 'calc_roc',
 'calc_rsi',
 'calc_rvalue',
 'calc_sar',
 'calc_shift',
 'calc_sign',
 'calc_slope',
 'calc_sma',
 'calc_stdev',
 'calc_step',
 'calc_stoch',
 'calc_streak',
 'calc_sum',
 'calc_tema',
 'calc_trange',
 'calc_tsf',
 'calc_typprice',
 'calc_updown',
 'calc_wclprice',
 'calc_wma']

In [8]:
from pprint import pformat

import importlib.util


def make_indicators(cnames=None):
    if cnames is None:
        cnames = core_functions()

    output = PRELUDE + "\n\n"

    fnames = []

    for cname in cnames:
        cfunc = getattr(core, cname)
        name = cname.removeprefix("calc_").upper()
        code = make_indicator(cfunc, name)
        fnames.append(name)
        output += code + "\n"

    # output += "__all__ = [name for name in dir() if name.isupper()]\n"
    xnames = pformat(fnames, width=75, compact=True, indent=4)
    xnames = xnames.replace("[", " ").replace("]", "")
    output += f"__all__ = [\n{xnames}\n]\n"

    return output

output = make_indicators()

def new_module(name: str, code: str = None):
    spec = importlib.util.spec_from_loader(name, None)
    module = importlib.util.module_from_spec(spec)
    if code:
        exec(code, module.__dict__)
    return module


indicators = new_module("mintalib.indicators", output)
indicators.__all__

['ABS',
 'ADX',
 'ALMA',
 'ATR',
 'AVGPRICE',
 'BBANDS',
 'BBP',
 'BBW',
 'BOP',
 'CCI',
 'CLAG',
 'CMF',
 'CROSSOVER',
 'CROSSUNDER',
 'CURVE',
 'DEMA',
 'DIFF',
 'DMI',
 'EMA',
 'EVAL',
 'EXP',
 'FLAG',
 'HMA',
 'KAMA',
 'KELTNER',
 'KER',
 'LAG',
 'LOG',
 'LROC',
 'MACD',
 'MACDV',
 'MAD',
 'MAV',
 'MAX',
 'MDI',
 'MFI',
 'MIDPRICE',
 'MIN',
 'NATR',
 'PDI',
 'PPO',
 'PRICE',
 'QSF',
 'RMA',
 'ROC',
 'RSI',
 'RVALUE',
 'SAR',
 'SHIFT',
 'SIGN',
 'SLOPE',
 'SMA',
 'STDEV',
 'STEP',
 'STOCH',
 'STREAK',
 'SUM',
 'TEMA',
 'TRANGE',
 'TSF',
 'TYPPRICE',
 'UPDOWN',
 'WCLPRICE',
 'WMA']

In [9]:
indicators.SMA(period=5, item='close') @ prices


date
1980-12-12           NaN
1980-12-15           NaN
1980-12-16           NaN
1980-12-17           NaN
1980-12-18      0.092060
                 ...    
2024-10-15    230.256000
2024-10-16    230.704001
2024-10-17    231.326001
2024-10-18    232.816000
2024-10-21    233.851999
Length: 11056, dtype: float64

In [10]:
indicators = new_module(f"{PACKAGE}.indicators", output)

outfile = PKGDIR / "indicators.py"

print(f"Updating {outfile.name} ...")

outfile.write_text(output)

Updating indicators.py ...


6330